In [9]:
from langchain_community.llms import Ollama
from langchain_community.utilities import SQLDatabase

In [7]:
model = Ollama(base_url="http://localhost:11434",model="llama3")

In [8]:
model.invoke("What are you")

"I am LLaMA, a large language model trained by a team of researcher at Meta AI. My primary function is to understand and respond to human input in the form of text prompts, using natural language processing (NLP) techniques.\n\nI'm designed to be conversational, so I can engage with users in a way that feels like talking to a person. I've been trained on a massive dataset of text from various sources, including books, articles, and online conversations.\n\nMy capabilities include:\n\n1. **Text generation**: I can generate human-like text based on the input prompt.\n2. **Conversational dialogue**: I can engage in natural-sounding conversations, using context and understanding to respond to user inputs.\n3. **Answering questions**: I can provide answers to factual questions or offer insights based on my training data.\n4. **Language translation**: I can translate text from one language to another (though I'm not perfect at this task yet).\n5. **Creative writing**: I can generate creative

In [12]:
db_user="root"
db_password="Sunnyraina$1"
db_host="localhost"
db_name="bankmanagementsystem"
db=SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)

mysql


In [13]:
print(db.get_usable_table_names())

['signup', 'signuptwo']


In [14]:
db.run("SELECT * FROM bankmanagementsystem.signup")

"[('0', 'Aryan Raina', 'Kamal Raina', 'Feb 4, 2003', 'Male', 'aryanraina2021@gmail.com', 'Unmarried', 'Lane no.22 Block no.130 Flat no.18 Jagti', 'Jammu', '181221', 'Jammu and kashmir'), ('0', 'Rishika Raina', 'Kamal Raina', 'Jul 27, 2005', 'Female', 'rishikaraina2005@gmail.com', 'Unmarried', 'Lane no.22 block no.130 Flat no.18 Jagti', 'Jammu', '181221', 'J&K'), ('6571', 'Kamal Raina', 'Janki Nath Raina', 'Jul 9, 2024', 'Male', 'kamalraina1969@gmail.com', 'married', 'ad ', 'dddd', '181221', 'dinin'), ('4374', 'rajan', 'bhan', 'Jul 4, 2024', 'Male', 'rajan44@gmail.com', 'married', 'Deathvalley ', 'Secret', '9000000', 'Unknown'), ('5015', 'rajan bhan', 'rajat bhan', 'Jul 1, 2024', 'Male', 'rajanbhan420@gmail.com', 'married', 'deathvalley', 'unknown', '910200', 'unknown')]"

In [15]:
from langchain.chains import create_sql_query_chain

In [16]:
chain= create_sql_query_chain(model,db)

In [24]:
response=chain.invoke({"question":"How many males are there in bankmanagementsystem.signup table"})

In [25]:
response

"Here is the SQL query to answer this question:\n\n```sql\nSELECT COUNT(*) AS total_males\nFROM signup\nWHERE gender = 'Male';\n``"

In [26]:
db.run(response)

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'Here is the SQL query to answer this question:\n\n```sql\nSELECT COUNT(*) AS tot...' at line 1")
[SQL: Here is the SQL query to answer this question:

```sql
SELECT COUNT(*) AS total_males
FROM signup
WHERE gender = 'Male';
``]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [29]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query=QuerySQLDataBaseTool(db=db)
write_query= create_sql_query_chain(model,db)
chain= write_query | execute_query
chain.invoke({"question":"How many tables are there"})

'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near \'Here\'s the answer:\\n\\nQuestion: How many tables are there\\nSQLQuery: SELECT COUN...\' at line 1")\n[SQL: Here\'s the answer:\n\nQuestion: How many tables are there\nSQLQuery: SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = \'default]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [30]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [31]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [32]:
chain=(
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query")|execute_query
    )
    | answer_prompt
    | model
    |StrOutputParser()
)

In [33]:
chain.invoke({"question":"How many tables are there"})

"A SQL error!\n\nThe error message is indicating that there's a syntax issue in the SQL query. Specifically, it seems that the `SQLQuery` section of the question contains the actual SQL code, which is not enclosed in backticks (``) or quotes (`''`). This is causing MariaDB to throw an error.\n\nTo fix this, we can simply enclose the entire SQL query in a string, like so:\n```sql\nSELECT COUNT(DISTINCT TABLE_NAME) \nFROM INFORMATION_SCHEMA.TABLES \nWHERE TABLE_SCHEMA = 'default';\n```\nNow, let's run this corrected query and answer the user question!\n\nAccording to the SQL result (assuming it's correct), there are ... [drumroll] ... **[insert number here]** tables in total!"

In [34]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit=SQLDatabaseToolkit(db=db,llm=model)

In [35]:
tools=toolkit.get_tools()

In [36]:
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000017DBA21AB70>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000017DBA21AB70>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000017DBA21AB70>),
 QuerySQLCheckerTool(description='Use this tool to 

In [37]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQL query to run without giving query into quotes  , then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [39]:
from langchain.agents import create_sql_agent

In [43]:
agent_executor= create_sql_agent(llm=model,toolkit=toolkit,system_message=system_message,verbose=True)

c:\python312\Lib\site-packages\langchain_community\agent_toolkits\sql\base.py:138: UserWarning: Received additional kwargs {'system_message': SystemMessage(content='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try

In [46]:
agent_executor.run("How many males are there in signup table")



> Entering new AgentExecutor chain...
Let's think this through...

Since we need to find out how many males are in the "signup" table, I think it would be best to execute a query that counts the number of rows where the gender is 'male'.

Action: sql_db_query
Action Input: `SELECT COUNT(*) FROM signup WHERE gender = 'male';Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '`SELECT COUNT(*) FROM signup WHERE gender = 'male'' at line 1")
[SQL: `SELECT COUNT(*) FROM signup WHERE gender = 'male';]
(Background on this error at: https://sqlalche.me/e/20/f405)It looks like the query I entered has a syntax error! The issue is likely due to the fact that I didn't enclose the column name "gender" in backticks (``), which are used for quoting table and column names.

Let me try again!

Action: sql_db_query
Action Input: `SELECT COUNT(*) FROM signup WHERE gender = '